# Necessary Upgrades
Run the cells in this section to make sure you have the latest version of sklearn and joblib.

Restart your kernel after installing.

In [1]:
## Update sklearn to prevent version mismatches
# !pip install sklearn --upgrade

In [2]:
## install joblib. This will be used to save your model. 
# !pip install joblib
## Restart your kernel after installing 

# Import Dependencies

In [3]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

import joblib

# Read the CSV

In [4]:
model_df = pd.read_csv("NBA_data/preprocessed_80-19_Wperc.csv", index_col = 0)
test_df = pd.read_csv("NBA_data/preprocessed_2020_Wperc.csv", index_col = 0)

In [5]:
model_df.head()
test_df.head()
model_df.columns

Index(['RK', 'Team', 'Year', 'Conf', 'playoffs_y_n', 'G', 'W', 'L', 'MP', 'FG',
       'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'W%',
       'MP_scalnorm', 'FG_scalnorm', 'FGA_scalnorm', 'FG%_scalnorm',
       '3P_scalnorm', '3PA_scalnorm', '3P%_scalnorm', '2P_scalnorm',
       '2PA_scalnorm', '2P%_scalnorm', 'FT_scalnorm', 'FTA_scalnorm',
       'FT%_scalnorm', 'ORB_scalnorm', 'DRB_scalnorm', 'AST_scalnorm',
       'STL_scalnorm', 'BLK_scalnorm', 'TOV_scalnorm', 'PF_scalnorm',
       'PTS_scalnorm', 'W%_scalnorm'],
      dtype='object')

# Train Test Split Based on Year and Conference (East)

## Select X and Y, Split (East)

In [6]:
drop_columns = ['playoffs_y_n', 'RK', 'Team', 'Year', 'Conf',
                'G', 'W', 'L', 
                'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%', 
                'ORB', 'DRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
                'W%',
               ]
df_east = model_df.loc[model_df["Conf"] == "East"]

train_M3E = df_east.loc[df_east["Year"] < 2019]
test_M3E = df_east.loc[df_east["Year"] >= 2019]

X_train_M3E = train_M3E.drop(columns=drop_columns)
X_test_M3E = test_M3E.drop(columns=drop_columns)

y_train_M3E = train_M3E[['playoffs_y_n']]
y_test_M3E = test_M3E[['playoffs_y_n']]

X_train_M3E.columns

Index(['MP_scalnorm', 'FG_scalnorm', 'FGA_scalnorm', 'FG%_scalnorm',
       '3P_scalnorm', '3PA_scalnorm', '3P%_scalnorm', '2P_scalnorm',
       '2PA_scalnorm', '2P%_scalnorm', 'FT_scalnorm', 'FTA_scalnorm',
       'FT%_scalnorm', 'ORB_scalnorm', 'DRB_scalnorm', 'AST_scalnorm',
       'STL_scalnorm', 'BLK_scalnorm', 'TOV_scalnorm', 'PF_scalnorm',
       'PTS_scalnorm', 'W%_scalnorm'],
      dtype='object')

## Model Scores (East)

In [7]:
print ("Logistic - East")
log_m3East = LogisticRegression()
log_m3East.fit(X_train_M3E, y_train_M3E)

log_east_train_score = log_m3East.score(X_train_M3E, y_train_M3E)
log_east_test_score = log_m3East.score(X_test_M3E, y_test_M3E)

print(f"Training Data Score: {log_east_train_score}")
print(f"Testing Data Score: {log_east_test_score}")

print ("RFC - East")
rfc_m3East= RandomForestClassifier()
rfc_m3East.fit(X_train_M3E, y_train_M3E)

rfc_east_train_score = rfc_m3East.score(X_train_M3E, y_train_M3E)
rfc_east_test_score = rfc_m3East.score(X_test_M3E, y_test_M3E)

print(f"Training Data Score: {rfc_east_train_score}")
print(f"Testing Data Score: {rfc_east_test_score}")

print ("SVC - East")
svc_m3East = SVC(kernel='linear', probability=True)
svc_m3East.fit(X_train_M3E, y_train_M3E)

svc_east_train_score = svc_m3East.score(X_train_M3E, y_train_M3E)
svc_east_test_score = svc_m3East.score(X_test_M3E, y_test_M3E)

print(f"Training Data Score: {svc_east_train_score}")
print(f"Testing Data Score: {svc_east_test_score}")

east_scores = [["Train", log_east_train_score, rfc_east_train_score, svc_east_train_score],
               ["Test", log_east_test_score, rfc_east_test_score, svc_east_test_score]]
df_east_scores = pd.DataFrame(east_scores, columns = ['Type','Logistic', 'RFC', 'SVM']) 
df_east_scores

Logistic - East
Training Data Score: 0.8847583643122676
Testing Data Score: 0.8666666666666667
RFC - East
Training Data Score: 1.0
Testing Data Score: 1.0
SVC - East
Training Data Score: 0.912639405204461
Testing Data Score: 1.0


,Type,Logistic,RFC,SVM
0,Train,0.884758,1.0,0.912639
1,Test,0.866667,1.0,1.000000


## Model Importances (East)

In [8]:
feature_names_M3E = X_train_M3E.columns

print ("Log - East")
coeff_log = log_m3East.coef_
importance_log = coeff_log[0]
abs_importance_log = abs(importance_log)
importance_list_log = list(zip(feature_names_M3E,importance_log, abs_importance_log))
importance_list_log.sort(key=lambda x:x[2], reverse = True)
for i in range (5):
    print (importance_list_log[i])

print ("RFC - East")
rfc_importances = rfc_m3East.feature_importances_
rfc_importances = sorted(zip(rfc_importances, feature_names_M3E), reverse=True)
for i in range (5):
    print (rfc_importances[i])

print ("SVM - East")
coeff_svm = svc_m3East.coef_
importance_svm = coeff_svm[0]
abs_importance_svm = abs(importance_svm)
importance_list_svm = list(zip(feature_names_M3E,importance_svm, abs_importance_svm))
importance_list_svm.sort(key=lambda x:x[2], reverse = True)
for i in range (5):
    print (importance_list_svm[i])

Log - East
('W%_scalnorm', 7.033619297315829, 7.033619297315829)
('DRB_scalnorm', 1.9580241770594453, 1.9580241770594453)
('2PA_scalnorm', -1.5785459357562277, 1.5785459357562277)
('TOV_scalnorm', -1.4250316324269237, 1.4250316324269237)
('STL_scalnorm', 1.2807571415238796, 1.2807571415238796)
RFC - East
(0.43583327295827123, 'W%_scalnorm')
(0.07808994670779486, '2P%_scalnorm')
(0.056896480622694784, 'FG%_scalnorm')
(0.04775756883639007, 'DRB_scalnorm')
(0.035423156009178725, '2PA_scalnorm')
SVM - East
('W%_scalnorm', 8.021022534374561, 8.021022534374561)
('2PA_scalnorm', -1.732631408904734, 1.732631408904734)
('DRB_scalnorm', 1.494551416713089, 1.494551416713089)
('STL_scalnorm', 1.1560112807047562, 1.1560112807047562)
('ORB_scalnorm', 1.0307051972002743, 1.0307051972002743)


## 2019 Predictions (East)

In [20]:
east_predictions_2019 = test_M3E[["Team", "Year", "playoffs_y_n"]]

log_probability = log_m3East.predict_proba(X_test_M3E)[:,1].tolist()
rfc_probability = rfc_m3East.predict_proba(X_test_M3E)[:,1].tolist()
svm_probability = svc_m3East.predict_proba(X_test_M3E)[:,1].tolist()

log_prediction = log_m3East.predict(X_test_M3E).tolist()
rfc_prediction = rfc_m3East.predict(X_test_M3E).tolist()
svm_prediction = svc_m3East.predict(X_test_M3E).tolist()

east_predictions_2019["prediction_log"] = log_prediction
east_predictions_2019["prob_log"] = log_probability
east_predictions_2019["prediction_rfc"] = rfc_prediction
east_predictions_2019["prob_rfc"] = rfc_probability
east_predictions_2019["prediction_svm"] = svm_prediction
east_predictions_2019["prob_svm"] = svm_probability

east_predictions_2019.loc[east_predictions_2019["playoffs_y_n"] == 1]
east_predictions_2019.sort_values("prob_log", ascending = False)
east_predictions_2019.sort_values("prob_rfc", ascending = False)
east_predictions_2019.sort_values("prob_svm", ascending = False)

,Team,Year,playoffs_y_n,prediction_log,prob_log,prediction_rfc,prob_rfc,prediction_svm,prob_svm
7,Toronto Raptors,2019,1,1,0.996944,1,0.95,1,1.000000
0,Milwaukee Bucks,2019,1,1,0.997113,1,1.00,1,1.000000
3,Philadelphia 76ers,2019,1,1,0.969245,1,0.96,1,0.999992
13,Boston Celtics,2019,1,1,0.941795,1,0.93,1,0.993032
21,Indiana Pacers,2019,1,1,0.736608,1,0.93,1,0.841960
23,Orlando Magic,2019,1,1,0.651999,1,0.84,1,0.747447
14,Brooklyn Nets,2019,1,1,0.602181,1,0.84,1,0.741037
24,Detroit Pistons,2019,1,0,0.399639,1,0.82,1,0.500000
25,Miami Heat,2019,0,0,0.436127,0,0.27,0,0.381760
18,Charlotte Hornets,2019,0,1,0.521789,0,0.27,0,0.375874


## 2020 Predictions (East)

In [34]:
test_df_east = test_df.loc[test_df["Conf"] == "East"]
east_predictions_2020 = test_df_east[["Team", "Year", "playoffs_y_n"]]

X_test_M3E_2020 = test_df_east.drop(columns=drop_columns)

log_probability = log_m3East.predict_proba(X_test_M3E_2020)[:,1].tolist()
rfc_probability = rfc_m3East.predict_proba(X_test_M3E_2020)[:,1].tolist()
svm_probability = svc_m3East.predict_proba(X_test_M3E_2020)[:,1].tolist()

log_prediction = log_m3East.predict(X_test_M3E_2020).tolist()
rfc_prediction = rfc_m3East.predict(X_test_M3E_2020).tolist()
svm_prediction = svc_m3East.predict(X_test_M3E_2020).tolist()

east_predictions_2020["prediction_log"] = log_prediction
east_predictions_2020["prob_log"] = log_probability
east_predictions_2020["prediction_rfc"] = rfc_prediction
east_predictions_2020["prob_rfc"] = rfc_probability
east_predictions_2020["prediction_svm"] = svm_prediction
east_predictions_2020["prob_svm"] = svm_probability

east_predictions_2020.loc[east_predictions_2020["playoffs_y_n"] == 1]
east_predictions_2020.sort_values("prob_log", ascending = False)
east_predictions_2020.sort_values("prob_rfc", ascending = False)
east_predictions_2020.sort_values("prob_svm", ascending = False)

,Team,Year,playoffs_y_n,prediction_log,prob_log,prediction_rfc,prob_rfc,prediction_svm,prob_svm
13,Toronto Raptors,2020,1,1,0.992227,1,0.93,1,1.000000
1,Milwaukee Bucks,2020,1,1,0.993599,1,0.98,1,1.000000
12,Boston Celtics,2020,1,1,0.990247,1,0.89,1,1.000000
17,Philadelphia 76ers,2020,0,1,0.941435,1,0.92,1,0.994868
10,Miami Heat,2020,0,1,0.894769,1,0.95,1,0.979169
21,Brooklyn Nets,2020,0,1,0.631325,1,0.80,1,0.784128
18,Indiana Pacers,2020,0,1,0.603234,1,0.88,1,0.700427
27,Orlando Magic,2020,0,1,0.668965,1,0.86,1,0.679588
29,Charlotte Hornets,2020,0,0,0.301318,0,0.22,0,0.183824
26,Chicago Bulls,2020,0,0,0.241418,0,0.03,0,0.141375


# Train Test Split Based on Year and Conference (West)

## Select X and Y, Split (West)

In [11]:
drop_columns = ['playoffs_y_n', 'RK', 'Team', 'Year', 'Conf',
                'G', 'W', 'L', 
                'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%', 
                'ORB', 'DRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
                'W%',
               ]

df_west = model_df.loc[model_df["Conf"] == "West"]

train_M3W = df_west.loc[df_west["Year"] < 2019]
test_M3W = df_west.loc[df_west["Year"] >= 2019]

X_train_M3W = train_M3W.drop(columns=drop_columns)
X_test_M3W = test_M3W.drop(columns=drop_columns)

y_train_M3W = train_M3W[['playoffs_y_n']]
y_test_M3W = test_M3W[['playoffs_y_n']]

X_train_M3W.columns

Index(['MP_scalnorm', 'FG_scalnorm', 'FGA_scalnorm', 'FG%_scalnorm',
       '3P_scalnorm', '3PA_scalnorm', '3P%_scalnorm', '2P_scalnorm',
       '2PA_scalnorm', '2P%_scalnorm', 'FT_scalnorm', 'FTA_scalnorm',
       'FT%_scalnorm', 'ORB_scalnorm', 'DRB_scalnorm', 'AST_scalnorm',
       'STL_scalnorm', 'BLK_scalnorm', 'TOV_scalnorm', 'PF_scalnorm',
       'PTS_scalnorm', 'W%_scalnorm'],
      dtype='object')

## Model Scores (West)

In [12]:
print ("Logistic - West")
log_m3West = LogisticRegression()
log_m3West.fit(X_train_M3W, y_train_M3W)

log_west_train_score = log_m3West.score(X_train_M3W, y_train_M3W)
log_west_test_score = log_m3West.score(X_test_M3W, y_test_M3W)

print(f"Training Data Score: {log_west_train_score}")
print(f"Testing Data Score: {log_west_test_score}")

print ("RFC - West")
rfc_m3West= RandomForestClassifier()
rfc_m3West.fit(X_train_M3W, y_train_M3W)

rfc_west_train_score = rfc_m3West.score(X_train_M3W, y_train_M3W)
rfc_west_test_score = rfc_m3West.score(X_test_M3W, y_test_M3W)

print(f"Training Data Score: {rfc_west_train_score}")
print(f"Testing Data Score: {rfc_west_test_score}")

print ("SVC - West")
svc_m3West = SVC(kernel='linear', probability=True)
svc_m3West.fit(X_train_M3W, y_train_M3W)

svc_west_train_score = svc_m3West.score(X_train_M3W, y_train_M3W)
svc_west_test_score = svc_m3West.score(X_test_M3W, y_test_M3W)

print(f"Training Data Score: {svc_west_train_score}")
print(f"Testing Data Score: {svc_west_test_score}")

west_scores = [["Train", log_west_train_score, rfc_west_train_score, svc_west_train_score],
               ["Test", log_west_test_score, rfc_west_test_score, svc_west_test_score]]
df_west_scores = pd.DataFrame(west_scores, columns = ['Type','Logistic', 'RFC', 'SVM']) 
df_west_scores

Logistic - West
Training Data Score: 0.9104477611940298
Testing Data Score: 0.9333333333333333
RFC - West
Training Data Score: 1.0
Testing Data Score: 1.0
SVC - West
Training Data Score: 0.9272388059701493
Testing Data Score: 0.8666666666666667


,Type,Logistic,RFC,SVM
0,Train,0.910448,1.0,0.927239
1,Test,0.933333,1.0,0.866667


## Model Importances (West)

In [13]:
feature_names_M3W = X_train_M3W.columns

print ("Log - West")
coeff_log = log_m3West.coef_
importance_log = coeff_log[0]
abs_importance_log = abs(importance_log)
importance_list_log = list(zip(feature_names_M3W,importance_log, abs_importance_log))
importance_list_log.sort(key=lambda x:x[2], reverse = True)
for i in range (5):
    print (importance_list_log[i])

print ("RFC - West")
rfc_importances = rfc_m3West.feature_importances_
rfc_importances = sorted(zip(rfc_importances, feature_names_M3W), reverse=True)
for i in range (5):
    print (rfc_importances[i])

print ("SVM - West")
coeff_svm = svc_m3West.coef_
importance_svm = coeff_svm[0]
abs_importance_svm = abs(importance_svm)
importance_list_svm = list(zip(feature_names_M3W,importance_svm, abs_importance_svm))
importance_list_svm.sort(key=lambda x:x[2], reverse = True)
for i in range (5):
    print (importance_list_svm[i])

Log - West
('W%_scalnorm', 6.42506744905737, 6.42506744905737)
('TOV_scalnorm', -2.254464058950554, 2.254464058950554)
('STL_scalnorm', 2.2420378546708193, 2.2420378546708193)
('DRB_scalnorm', 2.005517829107449, 2.005517829107449)
('FG%_scalnorm', 1.3992733659919658, 1.3992733659919658)
RFC - West
(0.3546450125074766, 'W%_scalnorm')
(0.09138252630470728, '2P%_scalnorm')
(0.07792211124627561, 'FG%_scalnorm')
(0.06802483325100935, 'DRB_scalnorm')
(0.0483234842049242, 'AST_scalnorm')
SVM - West
('W%_scalnorm', 6.842966657251069, 6.842966657251069)
('STL_scalnorm', 1.8323065850104658, 1.8323065850104658)
('DRB_scalnorm', 1.5284518521269648, 1.5284518521269648)
('TOV_scalnorm', -1.455380061410904, 1.455380061410904)
('FTA_scalnorm', 1.0642277732085965, 1.0642277732085965)


## 2019 Predictions (West)

In [31]:
west_predictions_2019 = test_M3W[["Team", "Year", "playoffs_y_n"]]

log_probability = log_m3West.predict_proba(X_test_M3W)[:,1].tolist()
rfc_probability = rfc_m3West.predict_proba(X_test_M3W)[:,1].tolist()
svm_probability = svc_m3West.predict_proba(X_test_M3W)[:,1].tolist()

log_prediction = log_m3West.predict(X_test_M3W).tolist()
rfc_prediction = rfc_m3West.predict(X_test_M3W).tolist()
svm_prediction = svc_m3West.predict(X_test_M3W).tolist()

west_predictions_2019["prediction_log"] = log_prediction
west_predictions_2019["prob_log"] = log_probability
west_predictions_2019["prediction_rfc"] = rfc_prediction
west_predictions_2019["prob_rfc"] = rfc_probability
west_predictions_2019["prediction_svm"] = svm_prediction
west_predictions_2019["prob_svm"] = svm_probability

west_predictions_2019.loc[west_predictions_2019["playoffs_y_n"] == 1]
west_predictions_2019.sort_values("prob_log", ascending = False)
west_predictions_2019.sort_values("prob_rfc", ascending = False)
west_predictions_2019.sort_values("prob_svm", ascending = False)

,Team,Year,playoffs_y_n,prediction_log,prob_log,prediction_rfc,prob_rfc,prediction_svm,prob_svm
19,Denver Nuggets,2019,1,1,0.954571,1,0.90,1,0.999995
5,Portland Trail Blazers,2019,1,1,0.908849,1,0.85,1,0.994637
16,Utah Jazz,2019,1,1,0.935694,1,0.93,1,0.993136
1,Golden State Warriors,2019,1,1,0.945788,1,0.99,1,0.990560
6,Oklahoma City Thunder,2019,1,1,0.790825,1,0.86,1,0.953320
4,Los Angeles Clippers,2019,1,1,0.720825,1,0.80,1,0.807532
10,Houston Rockets,2019,1,1,0.659084,1,0.84,1,0.768726
17,San Antonio Spurs,2019,1,1,0.720426,1,0.80,1,0.741008
8,Sacramento Kings,2019,0,0,0.439509,0,0.36,1,0.514594
2,New Orleans Pelicans,2019,0,1,0.621482,0,0.40,1,0.500000


## 2020 Predictions (West)

In [37]:
test_df_west = test_df.loc[test_df["Conf"] == "West"]
west_predictions_2020 = test_df_west[["Team", "Year", "playoffs_y_n"]]

X_test_M3W_2020 = test_df_west.drop(columns=drop_columns)

log_probability = log_m3West.predict_proba(X_test_M3W_2020)[:,1].tolist()
rfc_probability = rfc_m3West.predict_proba(X_test_M3W_2020)[:,1].tolist()
svm_probability = svc_m3West.predict_proba(X_test_M3W_2020)[:,1].tolist()

log_prediction = log_m3West.predict(X_test_M3W_2020).tolist()
rfc_prediction = rfc_m3West.predict(X_test_M3W_2020).tolist()
svm_prediction = svc_m3West.predict(X_test_M3W_2020).tolist()

west_predictions_2020["prediction_log"] = log_prediction
west_predictions_2020["prob_log"] = log_probability
west_predictions_2020["prediction_rfc"] = rfc_prediction
west_predictions_2020["prob_rfc"] = rfc_probability
west_predictions_2020["prediction_svm"] = svm_prediction
west_predictions_2020["prob_svm"] = svm_probability

west_predictions_2020.loc[west_predictions_2020["playoffs_y_n"] == 1]
west_predictions_2020.sort_values("prob_log", ascending = False)
west_predictions_2020.sort_values("prob_rfc", ascending = False)
west_predictions_2020.sort_values("prob_svm", ascending = False)

,Team,Year,playoffs_y_n,prediction_log,prob_log,prediction_rfc,prob_rfc,prediction_svm,prob_svm
14,Los Angeles Lakers,2020,1,1,0.974413,1,0.97,1,0.999999
15,Denver Nuggets,2020,0,1,0.953843,1,0.94,1,0.999991
6,Los Angeles Clippers,2020,0,1,0.885693,1,0.95,1,0.983818
20,Oklahoma City Thunder,2020,0,1,0.830703,1,0.89,1,0.963170
16,San Antonio Spurs,2020,0,1,0.872555,0,0.36,1,0.929425
0,Dallas Mavericks,2020,0,1,0.808951,1,0.92,1,0.844488
2,Houston Rockets,2020,0,1,0.691789,1,0.86,1,0.787660
8,Memphis Grizzlies,2020,0,1,0.674227,1,0.91,1,0.765964
9,Phoenix Suns,2020,0,1,0.616461,0,0.30,0,0.412370
19,Utah Jazz,2020,0,0,0.446241,1,0.89,0,0.362934


# Save the Model

In [16]:
east_log = 'models/east_log_Wperc.h5'
east_rfc = 'models/east_rfc_Wperc.h5'
east_svm = 'models/east_svm_Wperc.h5'

joblib.dump(log_m3East, east_log)
joblib.dump(rfc_m3East, east_rfc)
joblib.dump(svc_m3East, east_svm)

west_log = 'models/west_log_Wperc.h5'
west_rfc = 'models/west_rfc_Wperc.h5'
west_svm = 'models/west_svm_Wperc.h5'

joblib.dump(log_m3West, west_log)
joblib.dump(rfc_m3West, west_rfc)
joblib.dump(svc_m3West, west_svm)

['models/west_svm_Wperc.h5']

In [17]:
loaded_model = joblib.load("models/east_log_Wperc.h5")
print(f"{loaded_model.score(X_test_M3E, y_test_M3E)}")
loaded_model = joblib.load("models/east_rfc_Wperc.h5")
print(f"{loaded_model.score(X_test_M3E, y_test_M3E)}")
loaded_model = joblib.load("models/east_svm_Wperc.h5")
print(f"{loaded_model.score(X_test_M3E, y_test_M3E)}")

loaded_model = joblib.load("models/west_log_Wperc.h5")
print(f"{loaded_model.score(X_test_M3W, y_test_M3W)}")
loaded_model = joblib.load("models/west_rfc_Wperc.h5")
print(f"{loaded_model.score(X_test_M3W, y_test_M3W)}")
loaded_model = joblib.load("models/west_svm_Wperc.h5")
print(f"{loaded_model.score(X_test_M3W, y_test_M3W)}")

0.8666666666666667
1.0
1.0
0.9333333333333333
1.0
0.8666666666666667
